In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Toys_and_Games.json.gz

--2021-01-29 11:30:28--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Toys_and_Games.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 999020493 (953M) [application/octet-stream]
Saving to: ‘Toys_and_Games.json.gz’

Toys_and_Games.json 100%[===================>] 952.74M  19.7MB/s    in 48s     

2021-01-29 11:31:17 (19.7 MB/s) - ‘Toys_and_Games.json.gz’ saved [999020493/999020493]



In [ ]:
### load the meta data

data = []
with gzip.open('Toys_and_Games.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

8201231
{'overall': 2.0, 'vote': '12', 'verified': False, 'reviewTime': '09 22, 2016', 'reviewerID': 'A1IDMI31WEANAF', 'asin': '0020232233', 'reviewerName': 'Mackenzie Kent', 'reviewText': "When it comes to a DM's screen, the space on the screen itself is at an absolute premium. The fact that 50% of this space is wasted on art (and not terribly informative or needed art as well) makes it completely useless. The only reason that I gave it 2 stars and not 1 was that, technically speaking, it can at least still stand up to block your notes and dice rolls. Other than that, it drops the ball completely.", 'summary': 'The fact that 50% of this space is wasted on art (and not terribly informative or needed art ...', 'unixReviewTime': 1474502400}


In [ ]:
df = pd.DataFrame.from_dict(data)

print(len(df))

8201231


In [ ]:
df.head(2)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,image,style
0,2.0,12,False,"09 22, 2016",A1IDMI31WEANAF,0020232233,Mackenzie Kent,"When it comes to a DM's screen, the space on t...",The fact that 50% of this space is wasted on a...,1474502400,NaN,NaN
1,1.0,21,False,"09 18, 2016",A4BCEVVZ4Y3V3,0020232233,Jonathan Christian,An Open Letter to GaleForce9*:\n\nYour unpaint...,Another worthless Dungeon Master's screen from...,1474156800,NaN,NaN


### Take the columns "**overall**","**verified**","**reviewTime**","**asin**".
**WHY ??**


*   "overall" : ratings of product given by users
*   "verified": to filter only true verified ratings
*   reviewerID: user id
*   "asin": unique product id
*   "reviewTime": to take only few years of ratings 








In [ ]:
#take overall,verified,reviewTime,asin columns
df2 = df[['overall','verified','reviewerID','asin','reviewTime']]

In [ ]:
# see top 5 rows
df2.head()

,overall,verified,reviewerID,asin,reviewTime
0,2.0,False,A1IDMI31WEANAF,0020232233,"09 22, 2016"
1,1.0,False,A4BCEVVZ4Y3V3,0020232233,"09 18, 2016"
2,3.0,True,A2EZ9PY1IHHBX0,0020232233,"09 12, 2016"
3,5.0,True,A139PXTTC2LGHZ,0020232233,"03 2, 2017"
4,1.0,True,A3IB33V29XIL8O,0020232233,"02 8, 2017"


In [ ]:
#check is there any null values
df2.isnull().sum()

overall       0
verified      0
reviewerID    0
asin          0
reviewTime    0
dtype: int64

**1. overall**

In [ ]:
df2['overall']

0          2.0
1          1.0
2          3.0
3          5.0
4          1.0
          ... 
8201226    5.0
8201227    5.0
8201228    5.0
8201229    5.0
8201230    5.0
Name: overall, Length: 8201231, dtype: float64

In [ ]:
df2['overall'].value_counts()

5.0    5375966
4.0    1126574
1.0     707027
3.0     609097
2.0     382567
Name: overall, dtype: int64

**2. verified**

In [ ]:
df2['verified']

0          False
1          False
2           True
3           True
4           True
           ...  
8201226    False
8201227     True
8201228     True
8201229     True
8201230     True
Name: verified, Length: 8201231, dtype: bool

**OBSERVATIONS**

*   There are verified and not verified  ratings
*   We only need verified ratings

In [ ]:
#take only verified reviews
df3 = df2[df2['verified']== True]

In [ ]:
df3.head(3)

,overall,verified,reviewerID,asin,reviewTime
2,3.0,True,A2EZ9PY1IHHBX0,0020232233,"09 12, 2016"
3,5.0,True,A139PXTTC2LGHZ,0020232233,"03 2, 2017"
4,1.0,True,A3IB33V29XIL8O,0020232233,"02 8, 2017"


In [ ]:
df3['verified'].value_counts()

True    7231522
Name: verified, dtype: int64

**3.reviewerID**

In [ ]:
df3['reviewerID']

2          A2EZ9PY1IHHBX0
3          A139PXTTC2LGHZ
4          A3IB33V29XIL8O
5          A1J86V48S4KRJE
7          A2UKOWP9ICU416
                ...      
8201225    A3KM7Y7B1Q8J8K
8201227    A1KTVUVADLKWZO
8201228    A2QCA9OE62IPZ4
8201229    A3N28JAZYS4L9O
8201230    A2OGYZRYRSVQZJ
Name: reviewerID, Length: 7231522, dtype: object

In [ ]:
df3['reviewerID'].isnull().sum()

0

**OBSERVATIONS**

Based on metadata dataset we dont know there is any blank values in reviwerID column

In [ ]:
def remove_null_in_reviewerID(x):
  if x.isalnum() == True:
    return x
  else:
    return None

In [ ]:
df4 = df3.copy()
df4['reviewerID'] = df4['reviewerID'].apply(remove_null_in_reviewerID)

In [ ]:
df4['reviewerID'].isnull().sum()

0

**OBSERVATIONS**

No null values in reviewerID column

In [ ]:
df4.head(3)

,overall,verified,reviewerID,asin,reviewTime
2,3.0,True,A2EZ9PY1IHHBX0,0020232233,"09 12, 2016"
3,5.0,True,A139PXTTC2LGHZ,0020232233,"03 2, 2017"
4,1.0,True,A3IB33V29XIL8O,0020232233,"02 8, 2017"


**4.asin**

In [ ]:
df4['asin']

2          0020232233
3          0020232233
4          0020232233
5          0020232233
7          0020232233
              ...    
8201225    B01HJBAKIO
8201227    B01HJHA7GI
8201228    B01HJHA7GI
8201229    B01HJHA7GI
8201230    B01HJHA7GI
Name: asin, Length: 7231522, dtype: object

In [ ]:
df4['asin'].isnull().sum()

0

OBSERVATIONS

There may be chance of null values in this column bacause the price column shows zero null values but in that column there are null values exist

In [ ]:
def remove_null_in_asin(x):
  if x.isalnum() == True:
    return x
  else:
    return None

In [ ]:
df5 = df4.copy()
df5['asin'] = df5['asin'].apply(remove_null_in_asin)

In [ ]:
df5['asin'].isnull().sum()

0

**OBSERVATIONS**

There is no null values in asin column

**5.reviewTime**

In [ ]:
df5['reviewTime']

2          09 12, 2016
3           03 2, 2017
4           02 8, 2017
5          01 27, 2017
7          12 17, 2016
              ...     
8201225     01 4, 2018
8201227    01 23, 2016
8201228     02 8, 2015
8201229    12 31, 2014
8201230    11 11, 2014
Name: reviewTime, Length: 7231522, dtype: object

**OBSERVATIONS**

*   In column we have full date so we need only year

In [ ]:
x= '02 20, 2015'
int(x.split(',')[1])

2015

In [ ]:
# split the reviewTime, take only year
df5['reviewyear'] = df5['reviewTime'].apply(lambda x: int(x.split(',')[1]))

In [ ]:
df5.head(2)

,overall,verified,reviewerID,asin,reviewTime,reviewyear
2,3.0,True,A2EZ9PY1IHHBX0,0020232233,"09 12, 2016",2016
3,5.0,True,A139PXTTC2LGHZ,0020232233,"03 2, 2017",2017


In [ ]:
df5['reviewyear'].value_counts()

2016    1761006
2015    1557659
2017    1358707
2014     922624
2018     640764
2013     542079
2012     201234
2011     106902
2010      68658
2009      38881
2008      18415
2007       9871
2006       1916
2005       1454
2004        509
2003        347
2002        316
2001        150
2000         29
1999          1
Name: reviewyear, dtype: int64

In [ ]:
# consider the ratings from year 2018
df6 = df5[df5['reviewyear']==2018]

In [ ]:
df6.head(2)

,overall,verified,reviewerID,asin,reviewTime,reviewyear
589,5.0,True,AWIR3UBL405QJ,0692770445,"04 4, 2018",2018
590,5.0,True,A1DFLJ9TJ9PH80,0692770445,"04 3, 2018",2018


In [ ]:
df6['reviewyear'].value_counts()

2018    640764
Name: reviewyear, dtype: int64

In [ ]:
df6.shape

(640764, 6)

In [ ]:
df6.head()

,overall,verified,reviewerID,asin,reviewTime,reviewyear
589,5.0,True,AWIR3UBL405QJ,0692770445,"04 4, 2018",2018
590,5.0,True,A1DFLJ9TJ9PH80,0692770445,"04 3, 2018",2018
591,5.0,True,A1K1J2TG88SOH8,0692770445,"04 2, 2018",2018
592,5.0,True,AOID2AE5OGHQM,0692770445,"04 1, 2018",2018
593,1.0,True,A3ILHRAH8ZRCBD,0692770445,"04 1, 2018",2018


In [ ]:
#drop the "verified" , "reviewTime"
df7 = df6.drop(["verified","reviewTime","reviewyear"],axis=1)

In [ ]:
df7.head(3)

,overall,reviewerID,asin
589,5.0,AWIR3UBL405QJ,0692770445
590,5.0,A1DFLJ9TJ9PH80,0692770445
591,5.0,A1K1J2TG88SOH8,0692770445


In [ ]:
df7.rename(columns = {'overall':'ratings'}, inplace = True)

In [ ]:
df7.head(2)

,ratings,reviewerID,asin
589,5.0,AWIR3UBL405QJ,0692770445
590,5.0,A1DFLJ9TJ9PH80,0692770445


In [ ]:
df7.shape

(640764, 3)

In [ ]:
#total unique products
df7['asin'].nunique()

129254

In [ ]:
#verify there is no null values in dataframe
df7.isnull().sum()

ratings       0
reviewerID    0
asin          0
dtype: int64

In [ ]:
#convert dataframe to csv
df7.to_csv("toys and games ratings dataset.csv",index=False)